### 17장 딥러닝을 이용한 자연어 처리

#### 2. 단어의 원-핫 인코딩

- 배열 인덱스 0부터 시작 → 맨 앞에 0 추가

<img src="https://user-images.githubusercontent.com/90624848/155364703-777ef282-9c58-47f9-8b85-c503d6d51c46.PNG" width="600" align="left">

<img src="https://user-images.githubusercontent.com/90624848/155364708-e7154fa0-0799-43d2-ae8d-534aaa0be79b.PNG" width="600" align="left">

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer

text="오랫동안 꿈꾸는 이는 그 꿈을 닮아간다"

token = Tokenizer()
token.fit_on_texts([text])
print(token.word_index)

{'오랫동안': 1, '꿈꾸는': 2, '이는': 3, '그': 4, '꿈을': 5, '닮아간다': 6}


In [2]:
# texts_to_sequences(): 앞서 만들어진 토큰의 인덱스로만 채워진 새로운 배열 생성
x = token.texts_to_sequences([text])
print(x)

[[1, 2, 3, 4, 5, 6]]


In [3]:
# to_categorical() 함수를 사용해 원-핫 인코딩 과정 진행
from keras.utils import to_categorical

# 인덱스 수에 하나를 추가해서(맨 앞에 0이 추가되므로) 원-핫 인코딩 배열 만들기
word_size = len(token.word_index) +1
x = to_categorical(x, num_classes=word_size)

print(x)

[[[0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 1.]]]


Using TensorFlow backend.


#### 3. 단어 임베딩
- 원-핫 인코딩 그대로 사용 → 벡터의 길이가 너무 길어짐
- ```단어 임베딩(word embedding)```: 공간적 낭비 해결
    - 주어진 배열을 정해진 길이로 압축
    - 예시: 16차원 벡터 → 4차원 벡터   
     
        <img src="https://thebook.io/img/080228/252_1.jpg" width="500" align="left">   
        <br>
        <br>
        <br>
        <br>
        <br>
        <img src="https://thebook.io/img/080228/252_2.jpg" width="200" align="left">
        <br>
        <br>
        <br>
        <br>
        <br>
        <br>
    - 단어 임베딩으로 얻은 결과가 밀집된 정보를 가지고 있고공간의 낭비가 적음.   
        ∵ 각 단어 간의 유사도 계산
        - 유사도 계산: 오차 역전파 사용
    - ```Embedding()``` 함수 사용
        - 최소 2개의 매개변수 필요: 입력, 출력 크기
        - Embeeding(16, 4, input_length=2): 총 입력 단어 수는 16개이지만 매번 2개씩만 넣겠다.

In [4]:
from keras.models import Sequential
from keras.layers import Embedding

model = Sequential()
model.add(Embedding(16, 4)) # 입력될 총 단어 수는 16, 임베딩 후 출력되는 벡터 크기 4

#### 4. 텍스트를 읽고 긍정, 부정 예측하기

In [5]:
import numpy
import tensorflow as tf

from numpy import array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding

In [6]:
# 텍스트 리뷰 자료 지정
docs = ["너무 재밌네요","최고예요","참 잘 만든 영화예요","추천하고 싶은 영화입니다","한번 더 보고싶네요","글쎄요","별로예요","생각보다 지루하네요","연기가 어색해요","재미없어요"]

# 긍정 리뷰는 1, 부정 리뷰는 0으로 클래스를 지정합니다.
classes = array([1,1,1,1,1,0,0,0,0,0])

In [7]:
# 토큰화
token = Tokenizer()
token.fit_on_texts(docs)
print(token.word_index)
x = token.texts_to_sequences(docs)
print("\n리뷰 텍스트, 토큰화 결과:\n",  x)

{'너무': 1, '재밌네요': 2, '최고예요': 3, '참': 4, '잘': 5, '만든': 6, '영화예요': 7, '추천하고': 8, '싶은': 9, '영화입니다': 10, '한번': 11, '더': 12, '보고싶네요': 13, '글쎄요': 14, '별로예요': 15, '생각보다': 16, '지루하네요': 17, '연기가': 18, '어색해요': 19, '재미없어요': 20}

리뷰 텍스트, 토큰화 결과:
 [[1, 2], [3], [4, 5, 6, 7], [8, 9, 10], [11, 12, 13], [14], [15], [16, 17], [18, 19], [20]]


In [8]:
# 패딩, 서로 다른 길이의 데이터를 4로 맞춤
padded_x = pad_sequences(x, 4)
print("\n패딩 결과:\n", padded_x)


패딩 결과:
 [[ 0  0  1  2]
 [ 0  0  0  3]
 [ 4  5  6  7]
 [ 0  8  9 10]
 [ 0 11 12 13]
 [ 0  0  0 14]
 [ 0  0  0 15]
 [ 0  0 16 17]
 [ 0  0 18 19]
 [ 0  0  0 20]]


In [9]:
# 임베딩에 입력될 단어 수 지정
word_size = len(token.word_index) +1

# 단어 임베딩을 포함하여 딥러닝 모델을 만들고 결과 출력
model = Sequential()
model.add(Embedding(word_size, 8, input_length=4))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])
model.fit(padded_x, classes, epochs=20)

print("\n Accuracy: %.4f" % (model.evaluate(padded_x, classes)[1]))

Train on 10 samples
Epoch 1/20
10/10 [==============================] - 1s 55ms/sample - loss: 0.6844 - accuracy: 0.8000
Epoch 2/20
10/10 [==============================] - 0s 196us/sample - loss: 0.6829 - accuracy: 0.8000
Epoch 3/20
10/10 [==============================] - 0s 199us/sample - loss: 0.6813 - accuracy: 0.8000
Epoch 4/20
10/10 [==============================] - 0s 199us/sample - loss: 0.6798 - accuracy: 0.8000
Epoch 5/20
10/10 [==============================] - 0s 195us/sample - loss: 0.6783 - accuracy: 0.8000
Epoch 6/20
10/10 [==============================] - 0s 305us/sample - loss: 0.6767 - accuracy: 0.8000
Epoch 7/20
10/10 [==============================] - 0s 199us/sample - loss: 0.6752 - accuracy: 0.9000
Epoch 8/20
10/10 [==============================] - 0s 299us/sample - loss: 0.6737 - accuracy: 0.9000
Epoch 9/20
10/10 [==============================] - 0s 202us/sample - loss: 0.6721 - accuracy: 0.9000
Epoch 10/20
10/10 [==============================] - 0s 196us/s